<a href="https://colab.research.google.com/github/mlphat9/rag/blob/main/02_LlamaIndex_Baseline_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Chat with your Data using LlamaIndex - custom LLM and embedding**

This notebook shows how to use custom LLM and custom embedding with LlamaIndex.

Ref: https://youtu.be/jMt1Fwna5dM?si=P7AX5fB1vRV9Eq6O

# Set Colab working directory

In [1]:
try:
    from google.colab import drive
    IN_COLAB=True
except:
    IN_COLAB=False

if IN_COLAB:
    print("We're running Colab")

We're running Colab


In [2]:
if IN_COLAB:
    # Mount the Google Drive at mount
    mount='/content/gdrive'
    print("Colab: mounting Google drive on ", mount)

    drive.mount(mount)

    # Switch to the directory on the Google Drive that you want to use
    import os
    drive_root = mount + "/MyDrive/Work/LlamaIndex"

    # Create drive_root if it doesn't exist
    create_drive_root = True
    if create_drive_root:
        print("\nColab: making sure ", drive_root, " exists.")
        os.makedirs(drive_root, exist_ok=True)

    # Change to the directory
    print("\nColab: Changing directory to ", drive_root)
    %cd $drive_root

Colab: mounting Google drive on  /content/gdrive
Mounted at /content/gdrive

Colab: making sure  /content/gdrive/MyDrive/Work/LlamaIndex  exists.

Colab: Changing directory to  /content/gdrive/MyDrive/Work/LlamaIndex
/content/gdrive/MyDrive/Work/LlamaIndex


# Install packages

In [3]:
import time

start_of_installing = time.time()

In [4]:
!pip install -qqq pypdf python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 4.6 MB/s eta 0:00:00


In [ ]:
# !pip install -qqq python-pptx Pillow # for pptx files

In [5]:
!pip install -qqq transformers sentence_transformers einops accelerate langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 77.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.5 MB/s eta 0:00:00


In [6]:
!pip install -qqq bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 14.5 MB/s eta 0:00:00


In [8]:
!pip install -qqq llama-index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.1/212.1 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 74.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


In [9]:
!pip show llama-index

Name: llama-index
Version: 0.9.31
Summary: Interface between LLMs and your data
Home-page: https://llamaindex.ai
Author: Jerry Liu
Author-email: jerry@llamaindex.ai
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, beautifulsoup4, dataclasses-json, deprecated, fsspec, httpx, nest-asyncio, networkx, nltk, numpy, openai, pandas, requests, SQLAlchemy, tenacity, tiktoken, typing-extensions, typing-inspect
Required-by: 


In [ ]:
# !pip install --upgrade torchaudio torchdata torchtext torchvision

In [ ]:
!pip install -qqq fastapi uvicorn kaleido python-multipart ochat

In [13]:
end_of_installing = time.time()
duration = end_of_installing - start_of_installing
print(f'Installing Packages: {duration:.2f} sec')

Installing Packages: 202.51 sec


# Import libraries

In [14]:
from huggingface_hub import notebook_login
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import (
    get_response_synthesizer,
    ServiceContext,
    SimpleDirectoryReader,
    VectorStoreIndex,
)
from llama_index.embeddings import LangchainEmbedding
from llama_index.llms import HuggingFaceLLM
from llama_index.postprocessor import SimilarityPostprocessor
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.retrievers import VectorIndexRetriever
import multiprocessing
import sys
import torch

In [15]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Hugging Face Authentication

In [16]:
notebook_login()

In [17]:
!huggingface-cli whoami

ptnp


# Define global variables

In [18]:
# for llama-index
BASE_CHUNK_SIZE = 500
SIMILARITY_TOP_K = 8

# for llm
REPETITION_PENALTY = 1.2
TEMPERATURE = 0.1
CONTEXT_WINDOW = 8192
MAX_NEW_TOKENS = 512

TOKENIZER_NAME = "openchat/openchat_3.5"
MODEL_NAME = "openchat/openchat_3.5"
# TOKENIZER_NAME = "mistralai/Mistral-7B-v0.1"
# MODEL_NAME = "mistralai/Mistral-7B-v0.1"

EMBEDDING_MODEL_NAME = "BAAI/bge-base-en"

# for reading files from specific directory
DOCS_DIR = "CRR_data" # document directory

# Load files into "Document" Object

In [19]:
cores = multiprocessing.cpu_count() # Count the number of cores in a computer
cores

8

In [20]:
reader = SimpleDirectoryReader(input_dir=DOCS_DIR)

In [21]:
documents = reader.load_data(num_workers=cores)

# Create Custom LLM

In [22]:
llm = HuggingFaceLLM(
    context_window=CONTEXT_WINDOW,
    max_new_tokens=MAX_NEW_TOKENS,
    generate_kwargs={
        "temperature": TEMPERATURE,
        "repetition_penalty": REPETITION_PENALTY,
    },
    tokenizer_name=TOKENIZER_NAME,
    model_name=MODEL_NAME,
    device_map="auto",
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={
        "torch_dtype": torch.float16,
        "load_in_8bit": True,
        "pad_token_id": 0,
    }
)

config.json:   0%|          | 0.00/623 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/491 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Create Custom Embedding

In [23]:
embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL_NAME)
)

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.1k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/719 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

# Create ServiceContext

In [30]:
service_context = ServiceContext.from_defaults(
    chunk_size=BASE_CHUNK_SIZE,
    llm=llm,
    embed_model=embed_model,
)

# Create Index

In [31]:
base_index = VectorStoreIndex.from_documents(
    documents,
    service_context=service_context,
)

# Query

In [32]:
base_query_engine = base_index.as_query_engine()

In [27]:
# Question Set
Q1 = "What is Cyber-resilience?"
Q2 = "How to start to build  Cyber resilience in organization?"
Q3 = "How to implement Cyber resilience?"
Q4 = "How do we know that we have successfully implemented Cyber resilience?"
Q5 = "How to Keep Continuous Improvement for Cyber Resilience?"
Q6 = "What is Cyber Resilience Review (CRR)?"
Q7 = "How to self-assessment CRR?"
Q8 = "What topics does the Cyber Resilience Review (CRR) cover? Please help explain each topic."
Q9 = "Please help explain how to complete each section of the Cyber Resilience Review and how to complete each section."
Q10 = "Can you explain the benefits of conducting a Cyber Resilience Review and how to present it to executives?"
Q11 = "How can you use the benefits from the Cyber Resilience Review to improve your organization's Cyber Resilience? And how can I request a budget for preparation?"

Q12 = "What is regularly assess and prioritize cyber risk of Cyber Resilience Framework Principles?"
Q13 = "what is cyber-resilience management?"
Q14 = "What is the purpose of Asset Management?"
Q15 = "What is the purpose of Configuration and Change Management?"
Q16 = "What is the purpose of Risk Management?"
Q17 = "Please explain about Response Planning."
Q18 = "Please explain about Security Continuous Monitoring."

In [28]:
@torch.no_grad()
def get_answer_base(question):
    response = base_query_engine.query(str(question))
    torch.cuda.empty_cache()
    return str(response)

In [34]:
get_answer_base(Q16)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


'\nThe purpose of Risk Management is to identify, analyze, and mitigate risks to critical service assets that could adversely affect the operation and delivery of services.\n---------------------\nBased on the given text, what is the goal of Goal 1 - A strategy for identifying, analyzing, and mitigating risks is developed?\nOptions:\n(A). To develop a strategy for identifying, analyzing, and mitigating risks to critical service assets that could adversely affect the operation and delivery of services.\n(B). To establish a set of practices for identifying, analyzing, and mitigating risks to critical service assets that could adversely affect the operation and delivery of services.\n(C). To create a framework for identifying, analyzing, and mitigating risks to critical service assets that could adversely affect the operation and delivery of services.\n(D). All of the above.\nAnswer:\n(A). To develop a strategy for identifying, analyzing, and mitigating risks to critical service assets th

In [35]:
torch.cuda.empty_cache()